# Searching neural architectures by using successive halving

In [6]:
import json 
import os
import numpy as np
import torch 
import torch.nn as nn 
import torch.nn.functional as F 
from catalyst import dl, utils 
from catalyst.contrib.datasets import MNIST 
from sklearn import datasets 
from sklearn.metrics import log_loss 
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import MinMaxScaler 
from torch import nn as nn 
from torch import optim 
from torch.utils.data import DataLoader, TensorDataset 
torch.manual_seed(0)

In [7]:
class MLP(nn.Module):
    def __init__(self, input_layer_size, output_layer_size, layer_configuration, activation_type='relu'):
        super(MLP, self).__init__()
        self.fully_connected_layers = nn.ModuleDict()
        self.activation_type = activation_type
        hidden_layer_number = 0
        for hidden_layer_idx in range(len(layer_configuration)):
            if hidden_layer_idx == 0:
                self.fully_connected_layers[str(hidden_layer_number)] = nn.Linear(
                    input_layer_size,
                    layer_configuration[hidden_layer_idx]
                )
                hidden_layer_number += 1
            if hidden_layer_idx == len(layer_configuration) - 1:
                self.fully_connected_layers[str(hidden_layer_number)] = nn.Linear(
                    layer_configuration[hidden_layer_idx],
                    output_layer_size
                )
            else:
                self.fully_connected_layers[str(hidden_layer_number)] = nn.Linear(
                    layer_configuration[hidden_layer_idx],
                    layer_configuration[hidden_layer_idx+1]
                )
                hidden_layer_number += 1
                
    def forward(self, x):
        for fc_key in self.fully_connected_layers:
            x = self.fully_connected_layers[fc_key](x)
            if fc_key != str(len(self.fully_connected_layers) -1):
                x = F.relu(x)
            else:
                x = F.softmax(x)
        return x

In [8]:
def train_and_evaluate_mlp(
    trial_number, 
    layer_configuration,
    epochs, 
    input_layer_size,
    output_layer_size,
    load_on_stage_start=False,
    best_or_last='last',
    verbose=False
):
    criterion = nn.CrossEntropyLoss()
    runner = dl.SupervisedRunner(
        input_key="features", output_key="logits", target_key="targets", loss_key="loss"
    )
    model = MLP(
        input_layer_size=input_layer_size,
        layer_configuration=layer_configuration,
        output_layer_size=output_layer_size,
    )
    optimizer = optim.Adam(model.parameters(), lr=0.02)
    checkpoint_logdir = "logs/trial_{}".format(trial_number)
    runner.train(
        model=model, 
        criterion=criterion,
        optimizer=optimizer, 
        loaders=loaders,
        num_epochs=epochs,
        callbacks=[
            dl.CheckpointCallback(
                logdir=checkpoint_logdir,
                loader_key="valid",
                metric_key="loss",
                mode="all",
                load_on_stage_start="last_full" if load_on_stage_start else None,
            )
        ],
        logdir="./logs",
        valid_loader="valid",
        valid_metric="loss", minimize_valid_metric=True,
        verbose=verbose
    )
    with open(os.path.join(checkpoint_logdir, '_metrics.json'), 'r') as f:
        metrics = json.load(f)
    if best_or_last == 'last':
        valid_loss = metrics['last']['_score_']
    else:
        valid_loss = metrics['best']['valid']['loss']
    return valid_loss

In [9]:
def get_random_configurations(number_of_configurations, rng):
    layer_configurations = []
    for _ in range(number_of_configurations):
        layer_configuration = []
        number_of_hidden_layers = rng.randint(low=1, high=6)
        for _ in range(number_of_hidden_layers):
            layer_configuration.append(
            rng.randint(low=2, high=100)
            )
            layer_configurations.append(layer_configuration)
    layer_configurations = np.array(layer_configurations)
    return layer_configurations

In [10]:
iris = datasets.load_iris()
iris_input_dataset = iris['data']
target = torch.from_numpy(iris['target'])
scaler = MinMaxScaler()
scaler.fit(iris_input_dataset)
iris_input_dataset = torch.from_numpy(
    scaler.transform(iris_input_dataset)
).float()
(X_train, X_test, y_train, y_test) = train_test_split(
    iris_input_dataset, target, test_size=0.33, random_state=42
)
training_dataset = TensorDataset(X_train, y_train)
validation_dataset = TensorDataset(X_test, y_test)
train_loader = DataLoader(
    training_dataset, batch_size=10, num_workers=1
)
valid_loader = DataLoader(
    validation_dataset, batch_size=10, num_workers=1
)
loaders = {"train": train_loader, "valid": valid_loader}


In [35]:
rng = np.random.RandomState(1234)
number_of_configurations = 20
layer_configurations = get_random_configurations(
    number_of_configurations, rng
)
successive_halving_epochs = [5, 5, 5]


/tmp/ipykernel_14862/1714294016.py:11: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  layer_configurations = np.array(layer_configurations)


In [36]:
layer_configurations

array([list([40, 55, 78, 26]), list([40, 55, 78, 26]),
       list([40, 55, 78, 26]), list([40, 55, 78, 26]), list([17, 51]),
       list([17, 51]), list([28, 32]), list([28, 32]),
       list([32, 28, 60, 94]), list([32, 28, 60, 94]),
       list([32, 28, 60, 94]), list([32, 28, 60, 94]), list([75]),
       list([78, 39, 36]), list([78, 39, 36]), list([78, 39, 36]),
       list([69, 13, 2]), list([69, 13, 2]), list([69, 13, 2]),
       list([77, 82]), list([77, 82]), list([4, 21, 14, 67]),
       list([4, 21, 14, 67]), list([4, 21, 14, 67]),
       list([4, 21, 14, 67]), list([83, 16, 73, 62]),
       list([83, 16, 73, 62]), list([83, 16, 73, 62]),
       list([83, 16, 73, 62]), list([48, 30, 83]), list([48, 30, 83]),
       list([48, 30, 83]), list([98, 14, 71, 97, 33]),
       list([98, 14, 71, 97, 33]), list([98, 14, 71, 97, 33]),
       list([98, 14, 71, 97, 33]), list([98, 14, 71, 97, 33]),
       list([91, 86]), list([91, 86]), list([43]), list([58]),
       list([72, 58, 88, 46

In [46]:
trial_numbers = np.array(range(len(layer_configurations)))
for succesive_idx, successive_halving_epoch in enumerate(successive_halving_epochs):
    valid_losses = []
    for idx, layer_configuration in enumerate(layer_configurations):
        trial_number = trial_numbers[idx]
        valid_loss = train_and_evaluate_mlp(
            trial_number, 
            input_layer_size=iris_input_dataset.shape[1], 
            output_layer_size=3,
            layer_configuration=layer_configuration,
            epochs=successive_halving_epoch,
            load_on_stage_start=False if succesive_idx==0 else True
        )
        valid_losses.append(valid_loss)
    if succesive_idx != len(successive_halving_epochs) - 1:
        succesive_halved_configurations = np.argsort(valid_losses)[:int(len(valid_losses)/2)]
        layer_configurations = layer_configurations[
            succesive_halved_configurations
        ]
        trial_numbers = trial_numbers[
            succesive_halved_configurations
        ]


/tmp/ipykernel_14862/2433633264.py:32: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = F.softmax(x)


train (1/5) loss: 1.027400243282318 | loss/mean: 1.027400243282318 | loss/std: 0.0779187769744518 | lr: 0.02 | momentum: 0.9
valid (1/5) loss: 0.8113264918327332 | loss/mean: 0.8113264918327332 | loss/std: 0.08662349382978496 | lr: 0.02 | momentum: 0.9
* Epoch (1/5) 
train (2/5) loss: 0.8243094205856323 | loss/mean: 0.8243094205856323 | loss/std: 0.060180884949187455 | lr: 0.02 | momentum: 0.9
valid (2/5) loss: 0.7436487555503846 | loss/mean: 0.7436487555503846 | loss/std: 0.08704131286976208 | lr: 0.02 | momentum: 0.9
* Epoch (2/5) 
train (3/5) loss: 0.6817425072193144 | loss/mean: 0.6817425072193144 | loss/std: 0.07159089520789767 | lr: 0.02 | momentum: 0.9
valid (3/5) loss: 0.5746945381164551 | loss/mean: 0.5746945381164551 | loss/std: 0.02605368067605671 | lr: 0.02 | momentum: 0.9
* Epoch (3/5) 
train (4/5) loss: 0.6983911335468292 | loss/mean: 0.6983911335468292 | loss/std: 0.1832744057593382 | lr: 0.02 | momentum: 0.9
valid (4/5) loss: 0.7424887418746948 | loss/mean: 0.7424887418

valid (5/5) loss: 0.5785036325454712 | loss/mean: 0.5785036325454712 | loss/std: 0.030418574688565656 | lr: 0.02 | momentum: 0.9
* Epoch (5/5) 
Top best models:
logs/trial_5/train.5.pth	0.5785
train (1/5) loss: 1.0195854365825652 | loss/mean: 1.0195854365825652 | loss/std: 0.08269525133056098 | lr: 0.02 | momentum: 0.9
valid (1/5) loss: 0.8041297197341919 | loss/mean: 0.8041297197341919 | loss/std: 0.13818679676860227 | lr: 0.02 | momentum: 0.9
* Epoch (1/5) 
train (2/5) loss: 0.8240260422229767 | loss/mean: 0.8240260422229767 | loss/std: 0.047567963979279884 | lr: 0.02 | momentum: 0.9
valid (2/5) loss: 0.765532660484314 | loss/mean: 0.765532660484314 | loss/std: 0.10660979907838541 | lr: 0.02 | momentum: 0.9
* Epoch (2/5) 
train (3/5) loss: 0.765397322177887 | loss/mean: 0.765397322177887 | loss/std: 0.10430373544679697 | lr: 0.02 | momentum: 0.9
valid (3/5) loss: 0.5982219576835632 | loss/mean: 0.5982219576835632 | loss/std: 0.030304099478018758 | lr: 0.02 | momentum: 0.9
* Epoch (3/

train (5/5) loss: 0.6205336928367615 | loss/mean: 0.6205336928367615 | loss/std: 0.06386394934882783 | lr: 0.02 | momentum: 0.9
valid (5/5) loss: 0.5861230731010437 | loss/mean: 0.5861230731010437 | loss/std: 0.05499827930587826 | lr: 0.02 | momentum: 0.9
* Epoch (5/5) 
Top best models:
logs/trial_11/train.5.pth	0.5861
train (1/5) loss: 1.0067299783229828 | loss/mean: 1.0067299783229828 | loss/std: 0.08846333420405453 | lr: 0.02 | momentum: 0.9
valid (1/5) loss: 0.790772306919098 | loss/mean: 0.790772306919098 | loss/std: 0.12917237030046028 | lr: 0.02 | momentum: 0.9
* Epoch (1/5) 
train (2/5) loss: 0.763530445098877 | loss/mean: 0.763530445098877 | loss/std: 0.07420942254634531 | lr: 0.02 | momentum: 0.9
valid (2/5) loss: 0.7069389700889588 | loss/mean: 0.7069389700889588 | loss/std: 0.1059019504419259 | lr: 0.02 | momentum: 0.9
* Epoch (2/5) 
train (3/5) loss: 0.6836718142032623 | loss/mean: 0.6836718142032623 | loss/std: 0.053182049919354484 | lr: 0.02 | momentum: 0.9
valid (3/5) l

train (3/5) loss: 0.6638736128807068 | loss/mean: 0.6638736128807068 | loss/std: 0.09588428928968722 | lr: 0.02 | momentum: 0.9
valid (3/5) loss: 0.5710262298583985 | loss/mean: 0.5710262298583985 | loss/std: 0.039459398772705365 | lr: 0.02 | momentum: 0.9
* Epoch (3/5) 
train (4/5) loss: 0.6414228141307831 | loss/mean: 0.6414228141307831 | loss/std: 0.1041334886610407 | lr: 0.02 | momentum: 0.9
valid (4/5) loss: 0.5591137528419494 | loss/mean: 0.5591137528419494 | loss/std: 0.014152686473565086 | lr: 0.02 | momentum: 0.9
* Epoch (4/5) 
train (5/5) loss: 0.7585746884346009 | loss/mean: 0.7585746884346009 | loss/std: 0.14328182299098394 | lr: 0.02 | momentum: 0.9
valid (5/5) loss: 0.7714681267738343 | loss/mean: 0.7714681267738343 | loss/std: 0.13407366195673923 | lr: 0.02 | momentum: 0.9
* Epoch (5/5) 
Top best models:
logs/trial_2/train.4.pth	0.5591
=> Loading checkpoint logs/trial_3/last_full.pth
full checkpoint logs/trial_3/last_full.pth loaded (global epoch 5, stage train, epoch 5)

train (5/5) loss: 1.013823425769806 | loss/mean: 1.013823425769806 | loss/std: 0.17169892860617933 | lr: 0.02 | momentum: 0.9
valid (5/5) loss: 0.9318407058715821 | loss/mean: 0.9318407058715821 | loss/std: 0.18503469369154948 | lr: 0.02 | momentum: 0.9
* Epoch (5/5) 
Top best models:
logs/trial_0/train.2.pth	0.5528
=> Loading checkpoint logs/trial_1/last_full.pth
full checkpoint logs/trial_1/last_full.pth loaded (global epoch 10, stage train, epoch 5)
train (1/5) loss: 0.6725760400295258 | loss/mean: 0.6725760400295258 | loss/std: 0.07435665795370366 | lr: 0.02 | momentum: 0.9
valid (1/5) loss: 0.5893844366073608 | loss/mean: 0.5893844366073608 | loss/std: 0.04683240492347999 | lr: 0.02 | momentum: 0.9
* Epoch (1/5) 
train (2/5) loss: 0.5935502707958221 | loss/mean: 0.5935502707958221 | loss/std: 0.0635230975687885 | lr: 0.02 | momentum: 0.9
valid (2/5) loss: 0.552819037437439 | loss/mean: 0.552819037437439 | loss/std: 0.0014933981954153071 | lr: 0.02 | momentum: 0.9
* Epoch (2/5) 
tr

In [39]:
best_loss_idx = np.argmin(valid_losses)
best_layer_configuration = layer_configurations[best_loss_idx]
best_loss_trial_number = trial_numbers[best_loss_idx]


# Searching neural architectures by using Hyperband


In [48]:
import math


In [49]:
resource_per_conf = 30 # R
N = 3

In [51]:
r_i

1

In [52]:
s_max = int(math.log(resource_per_conf, N))
bracket_resource = (s_max + 1) * resource_per_conf 
bracket_best_valid_losses = []
bracket_best_layer_configuration = []
for s in range(s_max, -1, -1):
    number_of_configurations = int(
        (bracket_resource / resource_per_conf) * (N**s / (s+1))
    )
    r = resource_per_conf * N**-s
    layer_configurations = get_random_configurations(
        number_of_configurations, rng
    )
    trial_numbers = np.array(
        range(len(layer_configurations))
    )
    valid_losses = []
    for i in range(s+1):
        number_of_configurations_i = int(
        number_of_configurations * N**-i
        )
        r_i = int(r * N**i)
        valid_losses = []
        for idx, layer_configuration in enumerate(layer_configurations):
            trial_number = '{}_{}'.format(s, trial_numbers[idx])
            valid_loss = train_and_evaluate_mlp(
                trial_number, 
                input_layer_size=iris_input_dataset.shape[1], 
                output_layer_size=3,
                layer_configuration=layer_configuration,
                epochs=r_i, 
                load_on_stage_start=False if s==s_max else True
            )
            valid_losses.append(valid_loss)
            if succesive_idx != len(successive_halving_epochs) - 1:
                succesive_halved_configurations = np.argsort(
                valid_losses
                )[:int(number_of_configurations_i/N)]
                layer_configurations = layer_configurations[
                succesive_halved_configurations
                ]
                trial_numbers = trial_numbers[
                succesive_halved_configurations
                ]
                best_loss_idx = np.argmin(valid_losses)
                best_layer_configuration = layer_configurations[best_loss_idx]
                best_loss_trial_number = trial_numbers[best_loss_idx]
                bracket_best_valid_losses.append(
                    valid_losses[best_loss_idx]
                )
                bracket_best_layer_configuration.append(
                    best_layer_configuration
                )

/tmp/ipykernel_14862/1714294016.py:11: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  layer_configurations = np.array(layer_configurations)
/tmp/ipykernel_14862/2433633264.py:32: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = F.softmax(x)


train (1/1) loss: 1.051391726732254 | loss/mean: 1.051391726732254 | loss/std: 0.0539550765737218 | lr: 0.02 | momentum: 0.9
valid (1/1) loss: 0.8299834251403808 | loss/mean: 0.8299834251403808 | loss/std: 0.08110294077353315 | lr: 0.02 | momentum: 0.9
* Epoch (1/1) 
Top best models:
logs/trial_3_0/train.1.pth	0.8300
train (1/1) loss: 1.0814664721488954 | loss/mean: 1.0814664721488954 | loss/std: 0.027893235685592137 | lr: 0.02 | momentum: 0.9
valid (1/1) loss: 0.9067293524742126 | loss/mean: 0.9067293524742126 | loss/std: 0.07792100873397796 | lr: 0.02 | momentum: 0.9
* Epoch (1/1) 
Top best models:
logs/trial_3_1/train.1.pth	0.9067
train (1/1) loss: 1.0814664721488954 | loss/mean: 1.0814664721488954 | loss/std: 0.027893235685592137 | lr: 0.02 | momentum: 0.9
valid (1/1) loss: 0.9067293524742126 | loss/mean: 0.9067293524742126 | loss/std: 0.07792100873397796 | lr: 0.02 | momentum: 0.9
* Epoch (1/1) 
Top best models:
logs/trial_3_2/train.1.pth	0.9067
train (1/1) loss: 1.081466472148895

Top best models:
logs/trial_3_25/train.1.pth	0.9016
train (1/1) loss: 1.0620781064033507 | loss/mean: 1.0620781064033507 | loss/std: 0.07332028201210686 | lr: 0.02 | momentum: 0.9
valid (1/1) loss: 0.901595139503479 | loss/mean: 0.901595139503479 | loss/std: 0.09546978508254522 | lr: 0.02 | momentum: 0.9
* Epoch (1/1) 
Top best models:
logs/trial_3_26/train.1.pth	0.9016
train (1/1) loss: 1.0620781064033507 | loss/mean: 1.0620781064033507 | loss/std: 0.07332028201210686 | lr: 0.02 | momentum: 0.9
valid (1/1) loss: 0.901595139503479 | loss/mean: 0.901595139503479 | loss/std: 0.09546978508254522 | lr: 0.02 | momentum: 0.9
* Epoch (1/1) 
Top best models:
logs/trial_3_27/train.1.pth	0.9016
train (1/1) loss: 1.0634414911270142 | loss/mean: 1.0634414911270142 | loss/std: 0.022463030352864403 | lr: 0.02 | momentum: 0.9
valid (1/1) loss: 1.021214985847473 | loss/mean: 1.021214985847473 | loss/std: 0.00976961052483376 | lr: 0.02 | momentum: 0.9
* Epoch (1/1) 
Top best models:
logs/trial_3_28/tra

valid (1/1) loss: 0.9599687457084656 | loss/mean: 0.9599687457084656 | loss/std: 0.057690952672707616 | lr: 0.02 | momentum: 0.9
* Epoch (1/1) 
Top best models:
logs/trial_3_51/train.1.pth	0.9600
train (1/1) loss: 1.0636008858680728 | loss/mean: 1.0636008858680728 | loss/std: 0.030568292680545273 | lr: 0.02 | momentum: 0.9
valid (1/1) loss: 0.9630955457687378 | loss/mean: 0.9630955457687378 | loss/std: 0.031549133933960964 | lr: 0.02 | momentum: 0.9
* Epoch (1/1) 
Top best models:
logs/trial_3_52/train.1.pth	0.9631
train (1/1) loss: 1.0636008858680728 | loss/mean: 1.0636008858680728 | loss/std: 0.030568292680545273 | lr: 0.02 | momentum: 0.9
valid (1/1) loss: 0.9630955457687378 | loss/mean: 0.9630955457687378 | loss/std: 0.031549133933960964 | lr: 0.02 | momentum: 0.9
* Epoch (1/1) 
Top best models:
logs/trial_3_53/train.1.pth	0.9631
train (1/1) loss: 1.0025716722011566 | loss/mean: 1.0025716722011566 | loss/std: 0.08647023610162505 | lr: 0.02 | momentum: 0.9
valid (1/1) loss: 0.809821

train (1/1) loss: 1.0742929458618162 | loss/mean: 1.0742929458618162 | loss/std: 0.029582615433299916 | lr: 0.02 | momentum: 0.9
valid (1/1) loss: 0.954572606086731 | loss/mean: 0.954572606086731 | loss/std: 0.06602881935608881 | lr: 0.02 | momentum: 0.9
* Epoch (1/1) 
Top best models:
logs/trial_3_77/train.1.pth	0.9546
train (1/1) loss: 1.0742929458618162 | loss/mean: 1.0742929458618162 | loss/std: 0.029582615433299916 | lr: 0.02 | momentum: 0.9
valid (1/1) loss: 0.954572606086731 | loss/mean: 0.954572606086731 | loss/std: 0.06602881935608881 | lr: 0.02 | momentum: 0.9
* Epoch (1/1) 
Top best models:
logs/trial_3_78/train.1.pth	0.9546
train (1/1) loss: 1.0742929458618162 | loss/mean: 1.0742929458618162 | loss/std: 0.029582615433299916 | lr: 0.02 | momentum: 0.9
valid (1/1) loss: 0.954572606086731 | loss/mean: 0.954572606086731 | loss/std: 0.06602881935608881 | lr: 0.02 | momentum: 0.9
* Epoch (1/1) 
Top best models:
logs/trial_3_79/train.1.pth	0.9546
train (1/3) loss: 1.08146647214889

...

train (15/30) loss: 0.598220670223236 | loss/mean: 0.598220670223236 | loss/std: 0.06398814495617283 | lr: 0.02 | momentum: 0.9
valid (15/30) loss: 0.5899559020996094 | loss/mean: 0.5899559020996094 | loss/std: 0.03604295630924886 | lr: 0.02 | momentum: 0.9
* Epoch (15/30) 
train (16/30) loss: 0.57574502825737 | loss/mean: 0.57574502825737 | loss/std: 0.03081015807965006 | lr: 0.02 | momentum: 0.9
valid (16/30) loss: 0.5763995051383972 | loss/mean: 0.5763995051383972 | loss/std: 0.03837792699124953 | lr: 0.02 | momentum: 0.9
* Epoch (16/30) 
train (17/30) loss: 0.6031041443347931 | loss/mean: 0.6031041443347931 | loss/std: 0.06813155916244738 | lr: 0.02 | momentum: 0.9
valid (17/30) loss: 0.5879887342453003 | loss/mean: 0.5879887342453003 | loss/std: 0.037362155514783585 | lr: 0.02 | momentum: 0.9
* Epoch (17/30) 
train (18/30) loss: 0.5734049379825592 | loss/mean: 0.5734049379825592 | loss/std: 0.0353238780931436 | lr: 0.02 | momentum: 0.9
valid (18/30) loss: 0.5745045900344848 | loss

train (15/30) loss: 0.6851683974266052 | loss/mean: 0.6851683974266052 | loss/std: 0.11276370604079128 | lr: 0.02 | momentum: 0.9
valid (15/30) loss: 0.6282920360565185 | loss/mean: 0.6282920360565185 | loss/std: 0.07290204293038144 | lr: 0.02 | momentum: 0.9
* Epoch (15/30) 
train (16/30) loss: 0.6557808697223663 | loss/mean: 0.6557808697223663 | loss/std: 0.11425730044693222 | lr: 0.02 | momentum: 0.9
valid (16/30) loss: 0.5782842636108398 | loss/mean: 0.5782842636108398 | loss/std: 0.04853697598153971 | lr: 0.02 | momentum: 0.9
* Epoch (16/30) 
train (17/30) loss: 0.6280758082866669 | loss/mean: 0.6280758082866669 | loss/std: 0.07363829579593353 | lr: 0.02 | momentum: 0.9
valid (17/30) loss: 0.5785884022712707 | loss/mean: 0.5785884022712707 | loss/std: 0.031107004645365827 | lr: 0.02 | momentum: 0.9
* Epoch (17/30) 
train (18/30) loss: 0.5956905364990234 | loss/mean: 0.5956905364990234 | loss/std: 0.08610226871400462 | lr: 0.02 | momentum: 0.9
valid (18/30) loss: 0.5961543917655945

train (15/30) loss: 0.6851683974266052 | loss/mean: 0.6851683974266052 | loss/std: 0.11276370604079128 | lr: 0.02 | momentum: 0.9
valid (15/30) loss: 0.6282920360565185 | loss/mean: 0.6282920360565185 | loss/std: 0.07290204293038144 | lr: 0.02 | momentum: 0.9
* Epoch (15/30) 
train (16/30) loss: 0.6557808697223663 | loss/mean: 0.6557808697223663 | loss/std: 0.11425730044693222 | lr: 0.02 | momentum: 0.9
valid (16/30) loss: 0.5782842636108398 | loss/mean: 0.5782842636108398 | loss/std: 0.04853697598153971 | lr: 0.02 | momentum: 0.9
* Epoch (16/30) 
train (17/30) loss: 0.6280758082866669 | loss/mean: 0.6280758082866669 | loss/std: 0.07363829579593353 | lr: 0.02 | momentum: 0.9
valid (17/30) loss: 0.5785884022712707 | loss/mean: 0.5785884022712707 | loss/std: 0.031107004645365827 | lr: 0.02 | momentum: 0.9
* Epoch (17/30) 
train (18/30) loss: 0.5956905364990234 | loss/mean: 0.5956905364990234 | loss/std: 0.08610226871400462 | lr: 0.02 | momentum: 0.9
valid (18/30) loss: 0.5961543917655945

train (15/30) loss: 0.6851683974266052 | loss/mean: 0.6851683974266052 | loss/std: 0.11276370604079128 | lr: 0.02 | momentum: 0.9
valid (15/30) loss: 0.6282920360565185 | loss/mean: 0.6282920360565185 | loss/std: 0.07290204293038144 | lr: 0.02 | momentum: 0.9
* Epoch (15/30) 
train (16/30) loss: 0.6557808697223663 | loss/mean: 0.6557808697223663 | loss/std: 0.11425730044693222 | lr: 0.02 | momentum: 0.9
valid (16/30) loss: 0.5782842636108398 | loss/mean: 0.5782842636108398 | loss/std: 0.04853697598153971 | lr: 0.02 | momentum: 0.9
* Epoch (16/30) 
train (17/30) loss: 0.6280758082866669 | loss/mean: 0.6280758082866669 | loss/std: 0.07363829579593353 | lr: 0.02 | momentum: 0.9
valid (17/30) loss: 0.5785884022712707 | loss/mean: 0.5785884022712707 | loss/std: 0.031107004645365827 | lr: 0.02 | momentum: 0.9
* Epoch (17/30) 
train (18/30) loss: 0.5956905364990234 | loss/mean: 0.5956905364990234 | loss/std: 0.08610226871400462 | lr: 0.02 | momentum: 0.9
valid (18/30) loss: 0.5961543917655945

FileNotFoundError: Missing file 'logs/trial_2_0/last_full.pth'!

# Searching neural architectures by using Bayesian hyperparameter optimization

In [54]:
from sklearn.gaussian_process import GaussianProcessRegressor

In [55]:
def get_bayesian_optimization_input_features(layer_configurations):
    bo_input_features = []
    for layer_configuration in layer_configurations:
        bo_input_feature = layer_configuration + [0] * (6 - len(layer_configuration))
        bo_input_features.append(bo_input_feature)
    return bo_input_features

In [56]:
number_of_configurations = [100, 2000]
epochs_per_conf = 15
topk_models = 5


In [58]:
trial_numbers = np.array(
    range(len(layer_configurations))
)
trial_number = 0
model = None
best_valid_losses_per_iteration = []
best_configurations_per_iteration = []
overall_bo_input_features = []
overall_bo_valid_losses = []
for number_of_configuration in number_of_configurations:
    valid_losses = []
    layer_configurations = get_random_configurations(
        number_of_configuration, rng
    )
    if model:
        bo_input_features = get_bayesian_optimization_input_features(layer_configurations)
        predicted_valid_losses = model.predict(
            bo_input_features
        )
        top_k_idx = np.argsort(
            predicted_valid_losses
        )[:topk_models]
        layer_configurations = layer_configurations[
            top_k_idx
        ]
    for idx, layer_configuration in enumerate(layer_configurations):
        trial_identifier = 'bo_{}'.format(trial_number)
        valid_loss = train_and_evaluate_mlp(
            trial_number, 
            input_layer_size=iris_input_dataset.shape[1], 
            output_layer_size=3,
            layer_configuration=layer_configuration,
            epochs=epochs_per_conf,
            load_on_stage_start=False,
            best_or_last='best'
        )
        valid_losses.append(valid_loss)
        trial_number += 1 
    best_loss_idx = np.argmin(valid_losses)
    best_valid_losses_per_iteration.append(
        valid_losses[best_loss_idx]
    )
    best_configurations_per_iteration.append(
        layer_configurations[best_loss_idx]
    )
    bo_input_features = get_bayesian_optimization_input_features(layer_configurations)
    overall_bo_input_features.extend(bo_input_features)
    overall_bo_valid_losses.extend(valid_losses)
    model = GaussianProcessRegressor()
    model.fit(
        overall_bo_input_features, 
        overall_bo_valid_losses
    )


/tmp/ipykernel_14862/1714294016.py:11: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  layer_configurations = np.array(layer_configurations)
/tmp/ipykernel_14862/2433633264.py:32: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = F.softmax(x)


train (1/15) loss: 1.1105597257614135 | loss/mean: 1.1105597257614135 | loss/std: 0.026692359348913938 | lr: 0.02 | momentum: 0.9
valid (1/15) loss: 1.0984391450881958 | loss/mean: 1.0984391450881958 | loss/std: 0.01435408310852528 | lr: 0.02 | momentum: 0.9
* Epoch (1/15) 
train (2/15) loss: 1.0965941190719606 | loss/mean: 1.0965941190719606 | loss/std: 0.004006419870531935 | lr: 0.02 | momentum: 0.9
valid (2/15) loss: 1.1002785921096803 | loss/mean: 1.1002785921096803 | loss/std: 0.006304637312494726 | lr: 0.02 | momentum: 0.9
* Epoch (2/15) 
train (3/15) loss: 1.0976936101913455 | loss/mean: 1.0976936101913455 | loss/std: 0.00675443826719348 | lr: 0.02 | momentum: 0.9
valid (3/15) loss: 1.101915717124939 | loss/mean: 1.101915717124939 | loss/std: 0.011454164809258968 | lr: 0.02 | momentum: 0.9
* Epoch (3/15) 
train (4/15) loss: 1.0979388594627382 | loss/mean: 1.0979388594627382 | loss/std: 0.009922413939067985 | lr: 0.02 | momentum: 0.9
valid (4/15) loss: 1.1034156322479247 | loss/m

train (1/15) loss: 1.1161424040794372 | loss/mean: 1.1161424040794372 | loss/std: 0.0345314736577223 | lr: 0.02 | momentum: 0.9
valid (1/15) loss: 1.0982954502105713 | loss/mean: 1.0982954502105713 | loss/std: 0.01674794163506465 | lr: 0.02 | momentum: 0.9
* Epoch (1/15) 
train (2/15) loss: 1.096289575099945 | loss/mean: 1.096289575099945 | loss/std: 0.005365734867303785 | lr: 0.02 | momentum: 0.9
valid (2/15) loss: 1.1011442422866822 | loss/mean: 1.1011442422866822 | loss/std: 0.010008415581770297 | lr: 0.02 | momentum: 0.9
* Epoch (2/15) 
train (3/15) loss: 1.097359549999237 | loss/mean: 1.097359549999237 | loss/std: 0.007264686267846628 | lr: 0.02 | momentum: 0.9
valid (3/15) loss: 1.1023531198501586 | loss/mean: 1.1023531198501586 | loss/std: 0.01342612500355667 | lr: 0.02 | momentum: 0.9
* Epoch (3/15) 
train (4/15) loss: 1.0974648118019104 | loss/mean: 1.0974648118019104 | loss/std: 0.009157827212389606 | lr: 0.02 | momentum: 0.9
valid (4/15) loss: 1.1028610467910767 | loss/mean:

* Epoch (15/15) 
Top best models:
logs/trial_3/train.1.pth	1.0983
train (1/15) loss: 1.0001943647861482 | loss/mean: 1.0001943647861482 | loss/std: 0.08788550875166115 | lr: 0.02 | momentum: 0.9
valid (1/15) loss: 0.809182059764862 | loss/mean: 0.809182059764862 | loss/std: 0.13670873785775836 | lr: 0.02 | momentum: 0.9
* Epoch (1/15) 
train (2/15) loss: 0.7989143967628479 | loss/mean: 0.7989143967628479 | loss/std: 0.057386516439031594 | lr: 0.02 | momentum: 0.9
valid (2/15) loss: 0.6735683917999268 | loss/mean: 0.6735683917999268 | loss/std: 0.06531871852058117 | lr: 0.02 | momentum: 0.9
* Epoch (2/15) 
train (3/15) loss: 0.6491065323352814 | loss/mean: 0.6491065323352814 | loss/std: 0.04017658598129999 | lr: 0.02 | momentum: 0.9
valid (3/15) loss: 0.6028829336166381 | loss/mean: 0.6028829336166381 | loss/std: 0.05846507625450368 | lr: 0.02 | momentum: 0.9
* Epoch (3/15) 
train (4/15) loss: 0.6086212754249574 | loss/mean: 0.6086212754249574 | loss/std: 0.0517481148620719 | lr: 0.02 |

valid (15/15) loss: 0.587175726890564 | loss/mean: 0.587175726890564 | loss/std: 0.04325511093497118 | lr: 0.02 | momentum: 0.9
* Epoch (15/15) 
Top best models:
logs/trial_5/train.14.pth	0.5733
train (1/15) loss: 1.0001943647861482 | loss/mean: 1.0001943647861482 | loss/std: 0.08788550875166115 | lr: 0.02 | momentum: 0.9
valid (1/15) loss: 0.809182059764862 | loss/mean: 0.809182059764862 | loss/std: 0.13670873785775836 | lr: 0.02 | momentum: 0.9
* Epoch (1/15) 
train (2/15) loss: 0.7989143967628479 | loss/mean: 0.7989143967628479 | loss/std: 0.057386516439031594 | lr: 0.02 | momentum: 0.9
valid (2/15) loss: 0.6735683917999268 | loss/mean: 0.6735683917999268 | loss/std: 0.06531871852058117 | lr: 0.02 | momentum: 0.9
* Epoch (2/15) 
train (3/15) loss: 0.6491065323352814 | loss/mean: 0.6491065323352814 | loss/std: 0.04017658598129999 | lr: 0.02 | momentum: 0.9
valid (3/15) loss: 0.6028829336166381 | loss/mean: 0.6028829336166381 | loss/std: 0.05846507625450368 | lr: 0.02 | momentum: 0.9


...

train (15/15) loss: 0.9016146779060363 | loss/mean: 0.9016146779060363 | loss/std: 0.10290218801143018 | lr: 0.02 | momentum: 0.9
valid (15/15) loss: 0.8514445185661316 | loss/mean: 0.8514445185661316 | loss/std: 0.20203036440724503 | lr: 0.02 | momentum: 0.9
* Epoch (15/15) 
Top best models:
logs/trial_304/train.7.pth	0.5790


/tmp/ipykernel_14862/1714294016.py:11: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  layer_configurations = np.array(layer_configurations)
/home/dcek/anaconda3/envs/py310/lib/python3.10/site-packages/torch/nn/parallel/data_parallel.py:32: UserWarning: 
    There is an imbalance between your GPUs. You may want to exclude GPU 1 which
    has less than 75% of the memory or cores of GPU 0. You can do so by setting
    the device_ids argument to DataParallel, or by setting the CUDA_VISIBLE_DEVICES
    environment variable.
  warnings.warn(imbalance_warn.format(device_ids[min_pos], device_ids[max_pos]))
/tmp/ipykernel_14862/2433633264.py:32: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = F.softm

train (1/15) loss: 1.1011430621147154 | loss/mean: 1.1011430621147154 | loss/std: 0.031036565256725036 | lr: 0.02 | momentum: 0.9
valid (1/15) loss: 1.0617070198059082 | loss/mean: 1.0617070198059082 | loss/std: 0.019987888085999075 | lr: 0.02 | momentum: 0.9
* Epoch (1/15) 
train (2/15) loss: 0.978238570690155 | loss/mean: 0.978238570690155 | loss/std: 0.07164238623047349 | lr: 0.02 | momentum: 0.9
valid (2/15) loss: 0.8667152285575866 | loss/mean: 0.8667152285575866 | loss/std: 0.09410837889066719 | lr: 0.02 | momentum: 0.9
* Epoch (2/15) 
train (3/15) loss: 0.870817345380783 | loss/mean: 0.870817345380783 | loss/std: 0.06022854448628603 | lr: 0.02 | momentum: 0.9
valid (3/15) loss: 0.8083127737045288 | loss/mean: 0.8083127737045288 | loss/std: 0.07556640098349804 | lr: 0.02 | momentum: 0.9
* Epoch (3/15) 
train (4/15) loss: 0.8366216540336608 | loss/mean: 0.8366216540336608 | loss/std: 0.06241059809023939 | lr: 0.02 | momentum: 0.9
valid (4/15) loss: 0.8021223902702331 | loss/mean: 

train (1/15) loss: 1.008935296535492 | loss/mean: 1.008935296535492 | loss/std: 0.09285724086196197 | lr: 0.02 | momentum: 0.9
valid (1/15) loss: 0.795720887184143 | loss/mean: 0.795720887184143 | loss/std: 0.08769151811475366 | lr: 0.02 | momentum: 0.9
* Epoch (1/15) 
train (2/15) loss: 0.8331292033195495 | loss/mean: 0.8331292033195495 | loss/std: 0.06272755505899272 | lr: 0.02 | momentum: 0.9
valid (2/15) loss: 0.777813720703125 | loss/mean: 0.777813720703125 | loss/std: 0.11665238956192307 | lr: 0.02 | momentum: 0.9
* Epoch (2/15) 
train (3/15) loss: 0.8073484003543854 | loss/mean: 0.8073484003543854 | loss/std: 0.05729537457590455 | lr: 0.02 | momentum: 0.9
valid (3/15) loss: 0.7469719409942627 | loss/mean: 0.7469719409942627 | loss/std: 0.07920457565565465 | lr: 0.02 | momentum: 0.9
* Epoch (3/15) 
train (4/15) loss: 0.7320720374584199 | loss/mean: 0.7320720374584199 | loss/std: 0.03884741138031153 | lr: 0.02 | momentum: 0.9
valid (4/15) loss: 0.7039286613464355 | loss/mean: 0.70

train (1/15) loss: 1.0721407890319823 | loss/mean: 1.0721407890319823 | loss/std: 0.041071359627274144 | lr: 0.02 | momentum: 0.9
valid (1/15) loss: 0.8958200097084046 | loss/mean: 0.8958200097084046 | loss/std: 0.10316300315299337 | lr: 0.02 | momentum: 0.9
* Epoch (1/15) 
train (2/15) loss: 0.8800778687000275 | loss/mean: 0.8800778687000275 | loss/std: 0.07168536763741072 | lr: 0.02 | momentum: 0.9
valid (2/15) loss: 0.7991868495941162 | loss/mean: 0.7991868495941162 | loss/std: 0.11914281962715474 | lr: 0.02 | momentum: 0.9
* Epoch (2/15) 
train (3/15) loss: 0.8252937018871307 | loss/mean: 0.8252937018871307 | loss/std: 0.054673779411353866 | lr: 0.02 | momentum: 0.9
valid (3/15) loss: 0.777630877494812 | loss/mean: 0.777630877494812 | loss/std: 0.09805756038640213 | lr: 0.02 | momentum: 0.9
* Epoch (3/15) 
train (4/15) loss: 0.8027435421943664 | loss/mean: 0.8027435421943664 | loss/std: 0.055921830766869504 | lr: 0.02 | momentum: 0.9
valid (4/15) loss: 0.7610607028007508 | loss/mea